In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = "retina"

import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import finpy_tse as fpy
import tse_option as tso
import riskfolio as rp

In [ ]:
mpl.rcParams["figure.dpi"] = 150       # resolution in notebook/window
mpl.rcParams["savefig.dpi"] = 300      # resolution when saving

Config Cell

In [ ]:
J_START = "1394-09-18"
J_END   = "1404-09-18"

symbols = {
    "retap": "رتاپ",
    "alborz": "البرز",
}

sectors = {
    "auto": "خودرو",
    "base_metals": "فلزات اساسی",
}

helper functions

In [ ]:
def get_price_history(stock, start=J_START, end=J_END, adj=False):
    df = fpy.Get_Price_History(
        stock=stock,
        start_date=start,
        end_date=end,
        ignore_date=False,
        adjust_price=adj,
        show_weekday=False,
        double_date=False,
    )
    return df

def get_tse_option(symbols, start, end=None, j_date=True):
    return tso.download(
        symbols=symbols,
        j_date=j_date,
        start=start,
        end=end,
        adjust_price=True,
        drop_unadjusted=False,
    )

In [ ]:
#رتاپ
df1 = fpy.Get_Price_History(
    stock="رتاپ",
    start_date=J_START,
    end_date=J_END,
    ignore_date=False,
    adjust_price=True,   # or True if you want adjusted prices
    show_weekday=False,
    double_date=False
)

# https://main.tsetmc.com/SectorsSummary

In [ ]:
df5= fpy.Get_CWI_History(
    start_date='1395-01-01',
    end_date='1400-12-29',
    ignore_date=False,
    just_adj_close=False,
    show_weekday=False,
    double_date=False)

display(df5)

Total Index Information

In [ ]:
df_hamvazn=fpy.Get_EWI_History(
    start_date='1395-01-01',
    end_date='1400-12-29',
    ignore_date=False,
    just_adj_close=True,
    show_weekday=False,
    double_date=False)

display(df_hamvazn)

Total Weighted Index

In [ ]:
df52 = tso.download(symbols=["ارفع"], j_date=True, start="1400-07-20", end="1402-11-23", adjust_price=True, drop_unadjusted=False)
display(df52)

code to get information about a symbol

In [ ]:
#البرز
df2 = fpy.Get_Price_History(
    stock="البرز",
    start_date=J_START,
    end_date=J_END,
    ignore_date=False,
    adjust_price=True,
    show_weekday=False,
    double_date=False
)

In [ ]:
# Rename price column to stock names (use "Close" or "Adj Close" depending on df1/df2)
df1_prices = df1.rename(columns={"Close": "رتاپ"})
df2_prices = df2.rename(columns={"Close": "البرز"})

# Keep only the price columns and align on index
merged_df0 = pd.concat([df1_prices["رتاپ"], df2_prices["البرز"]], axis=1)

# Daily simple returns
returns = merged_df0.pct_change().dropna()

# Correlation between the two stocks as a single float
corr_value = returns["رتاپ"].corr(returns["البرز"])  # pandas gives a scalar

print(f"Correlation between رتاپ and البرز: {corr_value:.5f}")

In [ ]:
prices = merged_df0

summary = []
for col in prices.columns:
    s = prices[col].dropna()
    summary.append({
        "Stock": col,
        "StartDate": s.index[0],
        "EndDate": s.index[-1],
        "StartPrice": float(s.iloc[0]),
        "EndPrice": float(s.iloc[-1]),
        "TotalReturnPct": round((s.iloc[-1] / s.iloc[0] - 1) * 100, 2),
    })

summary_df = pd.DataFrame(summary)
summary_df

In [ ]:
STD_retap  = np.std(returns["رتاپ"])
STD_alborz = np.std(returns["البرز"])

print(f"Std Dev رتاپ : {STD_retap:.5f}")
print(f"Std Dev البرز : {STD_alborz:.5f}")

In [ ]:
# MAD for each stock (mean absolute deviation around its own mean)

retap_ret   = returns["رتاپ"]
alborz_ret  = returns["البرز"]
MAD_retap   = np.mean(np.abs(retap_ret  - retap_ret.mean()))
MAD_alborz  = np.mean(np.abs(alborz_ret - alborz_ret.mean()))

print(f"MAD رتاپ  : {MAD_retap:.5f}")
print(f"MAD البرز : {MAD_alborz:.5f}")

In [ ]:
# Returns for each stock

retap_ret  = returns["رتاپ"]
alborz_ret = returns["البرز"]

alpha = 0.95  # confidence level for CDaR

def calc_cdar(r, alpha=0.95):
    # 1) Cumulative wealth
    cum_wealth = (1 + r).cumprod()
    # 2) Running max of wealth
    running_max = cum_wealth.cummax()
    # 3) Drawdowns (fraction below peak)
    drawdowns = 1 - (cum_wealth / running_max)
    # 4) CDaR = mean of worst (1 - alpha) drawdowns
    threshold_dd = drawdowns.quantile(alpha)
    cdar = float(drawdowns[drawdowns >= threshold_dd].mean())
    return cdar

CDaR_retap  = calc_cdar(retap_ret,  alpha=alpha)
CDaR_alborz = calc_cdar(alborz_ret, alpha=alpha)

print(f"CDaR رتاپ  (alpha={alpha}) : {CDaR_retap:.5f}")
print(f"CDaR البرز (alpha={alpha}) : {CDaR_alborz:.5f}")

In [ ]:
def compute_basic_stats(returns: pd.Series) -> pd.Series:
    return pd.Series({
        "mean": returns.mean(),
        "std": returns.std(),
        "mad": np.mean(np.abs(returns - returns.mean())),
    })

def cdar(returns: pd.Series, alpha: float = 0.95) -> float:
    cum_wealth = (1 + returns).cumprod()
    running_max = cum_wealth.cummax()
    drawdowns = 1 - (cum_wealth / running_max)
    threshold = drawdowns.quantile(alpha)
    return drawdowns[drawdowns >= threshold].mean()

In [ ]:
stats = pd.DataFrame({
    "رتاپ": compute_basic_stats(returns["رتاپ"]),
    "البرز": compute_basic_stats(returns["البرز"]),
}).T

In [ ]:
df53 = tso.download(symbols=["شلرد","زماهان","نطرین","شپنا","برکت","بکاب"], j_date=True, start="1399-05-20", end=None, adjust_price=True, drop_unadjusted=False)

display(df53)

Information about multiple symbols at once

In [ ]:
#  df54 = tso.download(symbols=["خودرو و ساخت قطعات "], j_date=True, start="1399-05-20", end=None, adjust_price=True, drop_unadjusted=False)
# display(df54)

In [ ]:
df55 = tso.download(symbols=["خساپا "], j_date=True, start="1399-05-20", end=None, adjust_price=True, drop_unadjusted=False)
display(df55)

If we need information for multiple symbols together, we will write the above formula separately.

In [ ]:
std_dev = np.std(returns)
print(f"انحراف معیار: {std_dev}")

Calculating standard deviation is applicable to both industry indices and stocks.

In [ ]:
# Building the portfolio object    ### max RET
port = rp.Portfolio(returns=returns)

In [ ]:
# Select method and estimate input parameters:
# Method to estimate expected returns based on historical data
method_mu='hist'
# Method to estimate covariance matrix based on historical data
method_cov='hist'
# Estimate mean and covariance based on historical data
port.assets_stats(method_mu=method_mu, method_cov=method_cov)

# Estimate optimal portfolio:

model = 'Classic' # Could be Classic(historical), BL(Black Litterman), FM(Factor Model) or BL_FM(Black litterman with factors)
rm = 'MV' # Risk measure used, there are 13 available risk measures
obj = 'MinRisk' # Objective function, could be MinRisk, MaxRet, Utility or Sharpe
hist = True # Use historical scenarios for risk measures that depend on scenarios
rf = 0  # Risk free rate
l = 0 # Risk aversion factor, only useful when obj is 'Utility'

w = port.optimization(model=model, rm=rm, obj=obj, rf=rf, l=l, hist=hist)

display(w)

In [ ]:
from bidi.algorithm import get_display
import arabic_reshaper
from matplotlib.font_manager import FontProperties
import matplotlib.pyplot as plt

def reshape_rtl(text: str) -> str:
    if not text:
        return text
    return get_display(arabic_reshaper.reshape(text))

# If Vazir is in the same folder as your notebook:
vazir_font = FontProperties(fname="Vazir-Bold.ttf", size=14)

title_fa = "سبد بهینه (رتاپ & البرز)"  # Persian title

ax = rp.plot_pie(
    w=w,
    title=reshape_rtl(title_fa),
    others=0.05,
    nrow=25,
    cmap="tab20",
    height=6,
    width=10,
    ax=None
)

# --- Fix labels and legend to use RTL shaping + Persian font ---

# 1) All texts drawn directly on the axes (slice labels, percentages, etc.)
for txt in ax.texts:
    txt_str = txt.get_text()
    # If the text has Persian/Arabic characters, reshape it
    if any("\u0600" <= ch <= "\u06FF" for ch in txt_str):
        txt.set_text(reshape_rtl(txt_str))
    txt.set_fontproperties(vazir_font)

# 2) Legend labels (asset names)
leg = ax.get_legend()
if leg is not None:
    for t in leg.get_texts():
        t_str = t.get_text()
        if any("\u0600" <= ch <= "\u06FF" for ch in t_str):
            t.set_text(reshape_rtl(t_str))
        t.set_fontproperties(vazir_font)

# 3) Title again, with explicit font
ax.set_title(reshape_rtl(title_fa), fontproperties=vazir_font)

fig = ax.get_figure()
fig.savefig("optimum_portfolio_pie.png", dpi=300, bbox_inches="tight")
plt.show()

In [ ]:
# Mean and covariance from historical returns
mu = returns.mean()      # expected daily return for each stock
Sigma = returns.cov()    # daily covariance matrix

# Convert weights to NumPy vector
w_vec = w.values.reshape(-1, 1)

# Portfolio expected daily return and volatility
port_ret = float(mu @ w.values)                     # μ_p = wᵀ μ
port_vol = float(np.sqrt(w_vec.T @ Sigma.values @ w_vec))  # σ_p = √(wᵀ Σ w)

print(f"Portfolio expected daily return : {port_ret:.5f}")
print(f"Portfolio daily volatility (std): {port_vol:.5f}")
print("\nWeights:")
display(w)

In [ ]:
# 1) Make sure weights are a 1D Series
#    (w is usually a DataFrame with one column)
weights = w.squeeze()   # converts 1-column DataFrame -> Series

# 2) Portfolio daily returns time series
#    multiply each column's return by its weight and sum across columns
port_ret_series = returns.mul(weights, axis=1).sum(axis=1)

# 3) Portfolio expected daily return (scalar)
port_ret = float(port_ret_series.mean())

# 4) Portfolio MAD (mean absolute deviation of daily returns) - scalar
port_mad = float((port_ret_series - port_ret).abs().mean())

print(f"Portfolio expected daily return : {port_ret:.5f}")
print(f"Portfolio daily MAD risk        : {port_mad:.5f}")

In [ ]:
# 1) Make sure weights are a 1D Series
#    (w is usually a DataFrame with one column)
weights = w.squeeze()   # converts 1-column DataFrame -> Series

# 2) Portfolio daily returns time series
#    multiply each column's return by its weight and sum across columns
port_ret_series = returns.mul(weights, axis=1).sum(axis=1)

# 3) Portfolio expected daily return (scalar)
port_ret = float(port_ret_series.mean())

# 4) Portfolio CDaR (Conditional Drawdown at Risk)
alpha = 0.95  # confidence level for CDaR (95%)

# Cumulative wealth index
cum_wealth = (1 + port_ret_series).cumprod()

# Running maximum of wealth
running_max = cum_wealth.cummax()

# Drawdowns as *relative* loss from peak (0 = no DD, 0.2 = 20% below peak)
drawdowns = 1 - (cum_wealth / running_max)

# CDaR: average of drawdowns in the worst (1 - alpha) tail
threshold_dd = drawdowns.quantile(alpha)
cdar = float(drawdowns[drawdowns >= threshold_dd].mean())

print(f"Portfolio expected daily return     : {port_ret:.5f}")
print(f"Portfolio daily CDaR (alpha={alpha}): {cdar:.5f}")

Risk Measures available:

- 'MV': Standard Deviation  
- 'MAD': Mean Absolute Deviation  
- 'SLPM': Second Lower Partial Moment (Sortino Ratio)  
- 'MSV': Semi Standard Deviation  
- 'FLPM': First Lower Partial Moment (Omega Ratio)  
- 'CVaR': Conditional Value at Risk  
- 'EVaR': Entropic Value at Risk  
- 'WR': Worst Realization (Minimax)  
- 'MDD': Maximum Drawdown of uncompounded cumulative returns (Calmar Ratio)  
- 'ADD': Average Drawdown of uncompounded cumulative returns  
- 'CDaR': Conditional Drawdown at Risk of uncompounded cumulative returns  
- 'EDaR': Entropic Drawdown at Risk of uncompounded cumulative returns  
- 'UCI': Ulcer Index of uncompounded cumulative returns  

In [ ]:
# a=np.corrcoef( (returns["عيار"],returns["دلار"],returns["پخش"]))
# display(a)

Calculating correlation between industries and stocks

In [ ]:
start_date = "1394-09-18"
end_date   = "1404-09-18"

# 1) Download data
usd, auto, base_metals = (
    fpy.Get_USD_RIAL(start_date=start_date, end_date=end_date, ignore_date=False, show_weekday=False, double_date=False),
    fpy.Get_SectorIndex_History(sector="خودرو", start_date=start_date, end_date=end_date, ignore_date=False, just_adj_close=True, show_weekday=False, double_date=False),
    fpy.Get_SectorIndex_History(sector="فلزات اساسی", start_date=start_date, end_date=end_date, ignore_date=False, just_adj_close=True, show_weekday=False, double_date=False))

# ---------- KEY PART: fix index & duplicates ----------
def prepare_df(df):
    df = df.copy()
    # If J-Date column exists, use it as index (common in finpy_tse)
    if "J-Date" in df.columns:
        df = df.set_index("J-Date")
    # Drop duplicate index labels (keep the last observation)
    df = df[~df.index.duplicated(keep="last")]
    # (Optional) sort by index just in case
    df = df.sort_index()
    return df

usd         = prepare_df(usd)
auto        = prepare_df(auto)
base_metals = prepare_df(base_metals)

# ---------- 2) Build a combined price DataFrame ----------
usd_close  = usd["Close"].rename("USD")
auto_close = auto["Adj Close"].rename("Auto")
bm_close   = base_metals["Adj Close"].rename("BaseMetals")

df = pd.concat([usd_close, auto_close, bm_close], axis=1).dropna()

# ---------- 3) Returns & correlations ----------
returns = df.pct_change().dropna()

corr_auto = returns["USD"].corr(returns["Auto"])
corr_bm   = returns["USD"].corr(returns["BaseMetals"])

print(f"Correlation (USD vs Auto):        {corr_auto:.5f}")
print(f"Correlation (USD vs BaseMetals): {corr_bm:.5f}")

# ---------- 4) Simple bar chart ----------
plt.figure(figsize=(6, 4))
plt.bar(["Auto", "BaseMetals"], [corr_auto, corr_bm])
plt.ylabel("Correlation with USD (daily returns)")
plt.title("1394-09-18 to 1404-09-18: USD vs Auto & Base Metals")
plt.grid(axis="y", linestyle="--", alpha=0.5)
fig = ax.get_figure()
fig.savefig("optimum_portfolio_pie.png", dpi=300, bbox_inches="tight")
plt.show()

In [ ]:
# ----- 1) Build yearly series (Jalali years) -----
years = pd.Series(df.index, index=df.index).astype(str).str.slice(0, 4).astype(int)

# Use last value of each year (you can change to .mean())
df_yearly = df.groupby(years).last()
df_yearly = df_yearly.loc[1394:1404]

# Normalize to 100 at first year so trends are comparable
df_yearly_norm = df_yearly / df_yearly.iloc[0] * 100

# Mapping for Persian labels
series_name_map = {
    "USD": "دلار آزاد",
    "Auto": "شاخص گروه خودرو",
    "BaseMetals": "شاخص گروه فلزات اساسی",
}

# ----- 2) Plot with Persian (RTL-corrected) text -----
plt.figure(figsize=(10, 5))

for col in df_yearly_norm.columns:
    label_fa = series_name_map.get(col, col)
    plt.plot(
        df_yearly_norm.index,
        df_yearly_norm[col],
        marker="o",
        linewidth=2,
        label=reshape_rtl(label_fa),  # use RTL label
    )

# Axis labels & title in Persian
xlabel_fa = "سال (جلالی)"
ylabel_fa = "شاخص نرمال‌شده (پایه = ۱۰۰ در ۱۳۹۴)"
title_fa  = "روند سالانه دلار، خودرویی و فلزات اساسی (۱۳۹۴ تا ۱۴۰۴)"

plt.xlabel(reshape_rtl(xlabel_fa), fontproperties=vazir_font)
plt.ylabel(reshape_rtl(ylabel_fa), fontproperties=vazir_font)
plt.title(reshape_rtl(title_fa),  fontproperties=vazir_font)

plt.xticks(np.arange(1394, 1405, 1))
plt.grid(True, linestyle="--", alpha=0.4)

# Legend: fix font + RTL shaping (already reshaped in label, but set font)
leg = plt.legend()
for t in leg.get_texts():
    t.set_fontproperties(vazir_font)

plt.tight_layout()
fig = ax.get_figure()
fig.savefig("optimum_portfolio_pie.png", dpi=300, bbox_inches="tight")
plt.show()